In [3]:
import ccxt
import requests
from binance.client import Client
from binance.exceptions import BinanceAPIException
from datetime import datetime, timedelta
import time
import hashlib
import hmac
import json
import re
import os
import main


In [4]:
from main import *

In [8]:
def deribit_set_up(mode):
    logger.info(f"We are in: {mode} mode")
    client_id_testnet = "m_cYdGRs"
    client_secret_testnet = "CFMj4XERdBpYU5f-xGiEeM6q29WWPh78KbOllO8I5nI"

    client_id_realnet = "VUSrWKNX"
    client_secret_realnet = "CNIEmjiKy2p-h28O4Mda1QKD8hXJ3duA5rAODdLfvwE"

    if mode == "test":
        client_id = client_id_testnet
        client_secret = client_secret_testnet
        base_url = 'https://test.deribit.com/api/v2/' # test base url
    else:
        client_id = client_id_realnet
        client_secret = client_secret_realnet
        base_url = 'https://www.deribit.com/api/v2/'

    logging.info(f"The base url: {base_url}")

    client = ccxt.deribit({
        'apiKey': client_id,
        'secret': client_secret,
        'timeout': 150000,
    })
    if mode=="test":
        client.set_sandbox_mode(True)
    return client

In [12]:
client=deribit_set_up("test")

INFO:main:We are in: test mode


In [13]:
new_delta = float(information_for_options(client)[-2])
contract_size = float(information_for_options(client)[-1])

In [22]:
print(get_keys("binance_keys.json"))

('8mByJVFXdNbYbK9lYTaml4qEopUOXubb1JeIM4hBYbH1lxQEzXbyE7o3Wz42U9bj', 'EVXGqpetdxLzCJxi0Aqskb4bH7mNc35Jo2yw4b7O6ji3JU3nWQgHyHpbJii5x292')


In [26]:
def get_account_balances(symbol=None):
    api_key,api_secret=get_keys("binance_keys.json")
    client = Client(api_key, api_secret, testnet=False)
    time_offset=get_time_off_set("time.json")
    if time_offset:
        client.timestamp_offset = time_offset
    try:
        # Fetch account information
        account_info = client.get_account()

        # Extract balances
        balances = account_info['balances']
        balance_info = []

        # Iterate through each balance and append non-zero balances to the list
        if symbol==None:
            for balance in balances:
                asset = balance['asset']
                free_balance = float(balance['free'])
                locked_balance = float(balance['locked'])
                total_balance = free_balance + locked_balance

                if total_balance > 0:
                    balance_info.append({
                        'Asset': asset,
                        'Free': free_balance,
                        'Locked': locked_balance,
                        'Total': total_balance
                    })
        else: 
            for balance in balances:
                if balance['asset'] in symbol:
                    asset = balance['asset']
                    free_balance = float(balance['free'])
                    locked_balance = float(balance['locked'])
                    total_balance = free_balance + locked_balance

                    if total_balance > 0:
                        balance_info.append({
                            'Asset': asset,
                            'Free': free_balance,
                            'Locked': locked_balance,
                            'Total': total_balance
                        })

        return balance_info
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [35]:
liquid=get_account_balances("USDT")
print(liquid)
if liquid==[]:
    print("Account is empty")
elif liquid[0]['Free']<200:
    print("Insuffiecient asset")

[{'Asset': 'USDT', 'Free': 100.0, 'Locked': 0.0, 'Total': 100.0}]
Insuffiecient asset


In [12]:
def information_for_options():
    client=deribit_set_up("test")
    try:
        markets = client.fetch_markets()
        server_time = client.fetch_time()
        # Filter option markets
        option_markets = [market for market in markets if market['type'] == 'option']
        # print("Option markets:", option_markets)
        for i in option_markets:
            # print(i["info"])
            filtered_list = []
            expiry_timestamp = float(i['info']['expiration_timestamp'])/1000
            # Convert expiry timestamp to datetime
            expiry_datetime = datetime.utcfromtimestamp(expiry_timestamp) 

            # Calculate current datetime plus 20 weeks
            current_datetime = datetime.utcnow()
            twenty_weeks_later = current_datetime + timedelta(weeks=4)
            # Check if expiry datetime is greater than 20 weeks later
            if expiry_datetime <= twenty_weeks_later:
                filtered_list.append(i)
                # if call option and strike is not none, add to list
                second_filtered_list = []
                for item in filtered_list:
                    # if item["strike"] is not None and 
                    if item["optionType"] == "call" and item["strike"] is not None:

                        second_filtered_list.append(item)
                        # third_filtered_list = []
                        # # Loop through second filtered list and get contract with longest expiration
                        # for item in second_filtered_list:
                        #     expired_time = item["expiration_timestamp"]
                            
        #                     if item["expiration_timestamp"] is not None:
        #                         third_filtered_list.append(item)
                third_filtered_list = []
                expiry_timestamp_list = []
                if len(second_filtered_list) > 0:
                    longest_expiry_contract = max(second_filtered_list, key=lambda x: x['expiry'])

        # print(longest_expiry_contract)
        # Contract symbol
        symbol = longest_expiry_contract["id"]
        # Timestamp of contract            
        expire_time = float(longest_expiry_contract["info"]["expiration_timestamp"])
        # convert expiration timestamp to datetime
        expiry_datetime = datetime.utcfromtimestamp(expire_time/1000)
        # Get delta
        contract_delta = client.fetch_greeks(longest_expiry_contract["symbol"])
        delta = contract_delta["info"]["greeks"]["delta"]

        # Get contract size
        # Calculate share to purchase
        contract_size = longest_expiry_contract["info"]["contract_size"]
        share_to_purchase = float(delta) * float(contract_size)


        return expiry_datetime, share_to_purchase, symbol ,delta ,contract_size
    except Exception as e:
        logger.error(f"Failed to retrieve options information: {e}")
        print("ERROR INFO for option")
        return None, None, None, None, None

In [13]:
float(information_for_options()[-2])

INFO:main:We are in Deribit: test mode


0.09925

In [15]:
print(information_for_options())

INFO:main:We are in Deribit: test mode


(datetime.datetime(2024, 8, 9, 8, 0), 0.09938, 'ETH-9AUG24-4600-C', '0.09938', '1.0')
